In [196]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from datetime import datetime
import time
import requests
import pandas as pd
import numpy as np
import json

In [197]:
def generate_glassdoor_url(job_name, page_number):
    base_url = "https://www.glassdoor.co.in/Job/india-"
    job_name = job_name.replace(" ", "-").lower()
    srch = f"IL.0,5_IN115_KO6,{len(job_name)+6}"
    ip = f"_IP{page_number}"
    return (f"{base_url}{job_name}-jobs-SRCH_{srch}{ip}.htm")

In [198]:
def fetch_jobs(url):
    html_content = []
    
    options = Options()
    options.add_argument("window-size=1920,1080")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    driver.get(url)
    driver.minimize_window()
    
    try:
        driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
        time.sleep(2)
    except NoSuchElementException:
        try:
            driver.find_element(By.XPATH, "//*[@id='JAModal']/div/div[2]/span/svg").click()
            time.sleep(2)
        except:
            time.sleep(2)
            pass
    
    done = False
    while not done:
        job_cards = driver.find_elements(By.XPATH,"//article[@id='MainCol']//ul/li[@data-adv-type='GENERAL']")
        for card in job_cards:
            card.click()
            time.sleep(1)

            #Closes the signup prompt
            try:
                driver.find_element(By.XPATH,".//span[@class='SVGInline modal_closeIcon']").click()
                time.sleep(2)
            except NoSuchElementException:
                time.sleep(2)
                pass

            html_content.append(driver.page_source)
            done = True
    
#     time.sleep(100)
    return html_content

In [199]:
# import re
# from collections import namedtuple

# LanguageRegex = re.compile(
#     r"\b("
#     r"ada|algol|assembly|awk|bash|basic|c(\+\+|#)?|cobol|coffeescript|dart|delphi|"
#     r"elixir|elm|erlang|f#|fortran|go|groovy|haskell|html|java|javascript|julia|kotlin|"
#     r"lisp|lua|matlab|objective-c|ocaml|pascal|perl|php|powershell|prolog|python|r|"
#     r"ruby|rust|scala|scheme|scratch|shell|smalltalk|sql|swift|tcl|typescript|vb\.net|"
#     r"visual basic"
#     r")\b", re.IGNORECASE)

# FrameworkRegex = re.compile(
#     r"\b("
#     r"angular|aurelia|backbone\.js|bootstrap|bulma|cakephp|cherrypy|django|docker|"
#     r"ember\.js|express|fastapi|feathers\.js|flask|gatsby\.js|google cloud platform|"
#     r"grails|hapi\.js|hugo|ibm cloud|ionic|jest|jquery|koa|laravel|loopback|meteor|"
#     r"nestjs|next\.js|nuxt\.js|openstack|phoenix|polymer|pyramid|quasar|react\.js|"
#     r"redux|restify|ruby on rails|sails\.js|sanic|serverless|sinatra|socket\.io|"
#     r"strapi|styled-components|symfony|thunderbird|tornado|turboGears|uwsgi|vaadin|"
#     r"vue\.js|vuex|wxWidgets|yarn|yesod|zepto"
#     r")\b", re.IGNORECASE)

# DatabaseRegex = re.compile(
#     r"\b("
#     r"cassandra|couchbase|couchdb|firebase|mongodb|ms sql|mysql|oracle|postgresql|"
#     r"redis|sqlite"
#     r")\b", re.IGNORECASE)

# SkillsRegex = re.compile(
#     r"\b("
#     r"amazon web services|ansible|apache kafka|apache mesos|apache zookeeper|aws cloudformation|"
#     r"aws cloudwatch|aws lambda|azure|chef|circleci|cloudflare|docker|gcp|git|github|"
#     r"gitlab|jenkins|kubernetes|logstash|nginx|prometheus|puppet|redis|terraform|travis ci"
#     r")\b", re.IGNORECASE)

In [200]:
import re
from collections import namedtuple
LanguageRegex = re.compile(r"\b(ada|algol|assembly|awk|bash|c(\+\+|#)?|cobol|css|html5|coffeescript|d|dart|delphi|elixir|elm|erlang|f#|fortran|go|groovy|haskell|html|java|javascript|julia|kotlin|lisp|lua|matlab|objective-c|ocaml|pascal|perl|php|powershell|prolog|python|r|ruby|rust|scala|scheme|scratch|shell|smalltalk|sql|swift|tcl|typescript|vb\.net|visual basic)\b", re.IGNORECASE)

FrameworkRegex = re.compile(r"\b(angular|aurelia|backbone\.js|react|spacy|beautifulsoup|bootstrap|bulma|cakephp|cherrypy|django|docker|ember\.js|express|fastapi|feathers\.js|flask|gatsby\.js|google cloud platform|grails|hapi\.js|hugo|ibm cloud|ionic|jest|jquery|koa|laravel|loopback|meteor|nestjs|next\.js|nuxt\.js|openstack|phoenix|polymer|pyramid|quasar|react\.js|redux|restify|ruby on rails|sails\.js|sanic|serverless|sinatra|selenium|socket|socket.io|strapi|styled-components|symfony|thunderbird|tornado|turboGears|uwsgi|vaadin|vue|vuex|wxWidgets|yarn|yesod|zepto)\b", re.IGNORECASE)

DatabaseRegex = re.compile(r"\b(cassandra|couchbase|couchdb|firebase|mongodb|ms sql|mysql|oracle|postgresql|redis|sqlite)\b", re.IGNORECASE)

SkillsRegex= re.compile(r"\b(amazon web services|aws|ansible|apache kafka|apache mesos|apache zookeeper|aws cloudformation|aws cloudwatch|aws lambda|azure|chef|circleci|cloudflare|docker|gcp|git|github|gitlab|jenkins|kubernetes|logstash|nginx|prometheus|puppet|redis|terraform|travis ci)\b", re.IGNORECASE)

In [201]:
# import re
# from collections import namedtuple

# LanguageRegex = re.compile(
#     r"\b("
#     r"ada|algol|assembly|awk|bash|basic|c(\+\+|#)?|cobol|coffeescript|dart|delphi|"
#     r"elixir|elm|erlang|f#|fortran|go|groovy|haskell|html|java|javascript|julia|kotlin|"
#     r"lisp|lua|matlab|objective-c|ocaml|pascal|perl|php|powershell|prolog|python|r|"
#     r"ruby|rust|scala|scheme|scratch|shell|smalltalk|sql|swift|tcl|typescript|vb\.net|"
#     r"visual basic"
#     r")\b", re.IGNORECASE)

# FrameworkRegex = re.compile(
#     r"\b("
#     r"angular|aurelia|backbone\.js|bootstrap|bulma|cakephp|cherrypy|django|docker|"
#     r"ember\.js|express|fastapi|feathers\.js|flask|gatsby\.js|google cloud platform|"
#     r"grails|hapi\.js|hugo|ibm cloud|ionic|jest|jquery|koa|laravel|loopback|meteor|"
#     r"nestjs|next\.js|nuxt\.js|openstack|phoenix|polymer|pyramid|quasar|react\.js|"
#     r"redux|restify|ruby on rails|sails\.js|sanic|serverless|sinatra|socket\.io|"
#     r"strapi|styled-components|symfony|thunderbird|tornado|turboGears|uwsgi|vaadin|"
#     r"vue\.js|vuex|wxWidgets|yarn|yesod|zepto"
#     r")\b", re.IGNORECASE)

# DatabaseRegex = re.compile(
#     r"\b("
#     r"cassandra|couchbase|couchdb|firebase|mongodb|ms sql|mysql|oracle|postgresql|"
#     r"redis|sqlite"
#     r")\b", re.IGNORECASE)

# SkillsRegex = re.compile(
#     r"\b("
#     r"amazon web services|ansible|apache kafka|apache mesos|apache zookeeper|aws cloudformation|"
#     r"aws cloudwatch|aws lambda|azure|chef|circleci|cloudflare|docker|gcp|git|github|"
#     r"gitlab|jenkins|kubernetes|logstash|nginx|prometheus|puppet|redis|terraform|travis ci"
#     r")\b", re.IGNORECASE)

In [202]:
def process_skills(skills):
    # Get the list of languages
    languages = [lang[0] for lang in skills['languages']]
    
    # Remove empty strings and duplicates, ignoring case
    languages = list(set([lang.strip().lower() for lang in languages if lang.strip()]))
    
    # Combine the results into a dictionary
    return {
        'languages': languages,
        'frameworks': list(set([fr.strip().lower() for fr in skills['frameworks'] if fr.strip()])),
        'databases': list(set([db.strip().lower() for db in skills['databases'] if db.strip()])),
        'skills': list(set([skill.strip().lower() for skill in skills['skills'] if skill.strip()]))
}

In [204]:
import json
import re
from bs4 import BeautifulSoup

def parse_html_to_json(html,count,keyword):
    soup = BeautifulSoup(html, 'html.parser')

    company_name = ""
    job_title = ""
    location = ""
    job_description = ""
    avg_base_pay_est = ""
    company_rating = ""
    company_link = ""
    time_since_posted = ""
    company_skills = ""

    try:
        company_name = soup.find('div', {'class': 'css-87uc0g e1tk4kwz1'}).text
    except:
        company_name = "#N/A"
    try:
        job_title = soup.find('div', {'class': 'css-1vg6q84 e1tk4kwz4'}).text
    except:
        job_title = "#N/A"
    try:
        location = soup.find('div', {'class': 'css-56kyx5 e1tk4kwz5'}).text
    except:
        location = "#N/A"  
    try:
        avg_base_pay_est = soup.find('div', {'class': 'css-1bluz6i e2u4hf13'}).text
    except:
        avg_base_pay_est = "#N/A"  
    try:
        time_since_posted = soup.find('div', {'class': 'd-flex align-items-end pl-std css-1vfumx3'}).text
    except:
        time_since_posted = "#N/A"    
    try:
        company_rating = soup.find('span', {'class': 'css-1m5m32b e1tk4kwz2'}).text
    except:
        company_rating = "#N/A"

    try:   
        job_description = soup.find('div', {'class': 'css-jrwyhi e856ufb5'}).text
#         job_description = soup.text
    
        language_matches = LanguageRegex.findall(job_description)
        framework_matches = FrameworkRegex.findall(job_description)
        database_matches = DatabaseRegex.findall(job_description)
        skills_matches = SkillsRegex.findall(job_description)

        company_skills = {
            'languages': language_matches,
            'frameworks': framework_matches,
            'databases': database_matches,
            'skills': skills_matches
        }
        company_skills = process_skills(company_skills)  
        ctr=0
        for i in company_skills.values():
            for j in i:
                ctr+=1
        if ctr==0:
            company_skills="N/A"
        # debug here for 0 skills
    except:
        job_description = "#N/A"
    a_tags = soup.find_all('a', {'class': 'jobLink css-1rd3saf eigr9kq2'}) 
    try:
        company_link="https://glassdoor.co.in"+a_tags[count]["href"]
    except:
        company_link="#N/A"     
        

    # create dictionary object from extracted data
    job_data = {
        'keyword':keyword,
        'company_name': company_name,
        'job_title': job_title,
        'location': location,
        'job_description': job_description,
        'avg_base_pay_est': avg_base_pay_est,
        'company_rating': company_rating,
        'company_link': company_link,
        'time_since_posted': time_since_posted,
        'company_skills': company_skills
    }

    return json.dumps(job_data)


In [205]:
def scrape_job_results(JOB_NAME, NO_OF_PAGES):
    URLS = [generate_glassdoor_url(JOB_NAME, i) for i in range(1,NO_OF_PAGES+1)]

    json_list = []

    for url in URLS:
        html_content = fetch_jobs(url)
        for i in range(30):
            json_list.append(parse_html_to_json(html_content[i], i,JOB_NAME))
            
    return json_list

In [206]:
# ScrapeJobsList = ['Software Engineer', 'Data Scientist', 'Product Manager']

ScrapeJobsList = ['React Developer', 'Data Analyst','Nodejs Developer', 'UI/UX Developer',\
             'Django Developer', 'Flutter Developer', 'Java Developer',\
             'Frontend Developer', 'Full-Stack Developer', 'DevOps Engineer',\
             'Cloud Solutions Architect', 'Machine Learning Engineer',\
             'Artificial Intelligence Specialist', 'Cybersecurity Analyst',\
             'Network Engineer', 'Database Administrator', 'IT Support Specialist',\
             'Python Developer', 'R&D Engineer']

In [207]:
def SCRAPE_ALL_JOB_RESULTS(ScrapeJobsList, NO_OF_PAGES):
    final_json = {}
    for job in ScrapeJobsList:
        final_json[job] = scrape_job_results(job, NO_OF_PAGES)
    return final_json

In [208]:
final_json = SCRAPE_ALL_JOB_RESULTS(ScrapeJobsList[:2], 2)

/tmp/ipykernel_91848/961610879.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [209]:
len(final_json)

2

In [210]:
final_json

{'React Developer': ['{"keyword": "React Developer", "company_name": "FutureSoft India Pvt Ltd", "job_title": "HTML Developer", "location": "Gurgaon", "job_description": "should possess excellent skills in HTML5, CSS, JavaScript, and jQuery, React, Angular as well as experience with Adobe Photoshop and other design software. You should also have a strong understanding of browser compatibility issues and be able to troubleshoot problems with web pages or apps. Additionally, you should be familiar with agile methodology and be able to work within a team environment to meet tight deadlines.Duties and Responsibilities\\u00b7 Create high-quality HTML files based on the latest HTML5, CSS3, W3C standards, JavaScript, React and Angular.\\u00b7 Create web applications that match designer visions at a range of screen resolutions.\\u00b7 Optimize front end part of the applications for maximum speedJob Type: Full-timeSalary: \\u20b9400,000.00 - \\u20b91,200,000.00 per yearBenefits:Health insurance

In [211]:
with open('trying_out_all_20_jobs_response.json', 'w') as f:
    json.dump(final_json, f)

In [185]:
f = open('trying_out_all_20_jobs_response.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [212]:
final_json.keys()

dict_keys(['React Developer', 'Data Analyst'])

In [213]:
for i in final_json.keys():
    for j in range(len(final_json[i])):
        try:
            print(json.loads(final_json[i][j]))
        except:
            print("error")
    

{'keyword': 'React Developer', 'company_name': 'FutureSoft India Pvt Ltd', 'job_title': 'HTML Developer', 'location': 'Gurgaon', 'job_description': "should possess excellent skills in HTML5, CSS, JavaScript, and jQuery, React, Angular as well as experience with Adobe Photoshop and other design software. You should also have a strong understanding of browser compatibility issues and be able to troubleshoot problems with web pages or apps. Additionally, you should be familiar with agile methodology and be able to work within a team environment to meet tight deadlines.Duties and Responsibilities· Create high-quality HTML files based on the latest HTML5, CSS3, W3C standards, JavaScript, React and Angular.· Create web applications that match designer visions at a range of screen resolutions.· Optimize front end part of the applications for maximum speedJob Type: Full-timeSalary: ₹400,000.00 - ₹1,200,000.00 per yearBenefits:Health insuranceSchedule:Day shiftMonday to FridayAbility to commute

In [214]:
job_array=[]
for i in final_json.keys():
    for j in range(len(final_json[i])):

        job_array.append(json.loads(final_json[i][j]))


In [215]:
count=0
for i in job_array:
    if i["company_link"] =="#N/A":
        count+=1

In [216]:
count

60

In [217]:
jobs=pd.DataFrame(job_array)

In [221]:
len(jobs)

120

In [220]:
jobs.iloc[60:120]

,keyword,company_name,job_title,location,job_description,avg_base_pay_est,company_rating,company_link,time_since_posted,company_skills
60,Data Analyst,Larsen & Toubro Limited3.4,Data Analyst (Tableau/ SQL),Calcutta,Tableau Desktop - interactive advanced Dashboa...,"₹3,20,550 /yr (est.)",3.4,#N/A,#N/A,"{'languages': ['r', 'python', 'sql', 'powershe..."
61,Data Analyst,Adtitude Digital,Data Analyst,Remote,Data Analyst / Business AnalystJob Type: Full-...,"₹7,25,000 /yr (est.)",#N/A,#N/A,#N/A,"{'languages': ['sql'], 'frameworks': [], 'data..."
62,Data Analyst,Myntra.com4.2,Data Analyst,Bangalore,Data Analyst\n\nTeam: Brand Accelerator\nRoles...,"₹7,53,846 /yr (est.)",4.2,#N/A,#N/A,"{'languages': ['r', 'python', 'sql'], 'framewo..."
63,Data Analyst,EY3.9,Data Analyst,Bangalore,"At EY, you’ll have the chance to build a caree...","₹4,56,747 /yr (est.)",3.9,#N/A,#N/A,N/A
64,Data Analyst,iCRC3.9,Data Analyst,Noida,We are looking for Data Analyst with 8+ years ...,"₹4,16,294 /yr (est.)",3.9,#N/A,#N/A,"{'languages': ['python', 'sql'], 'frameworks':..."
65,Data Analyst,Worley3.7,Graduate Data Analyst,Mumbai,Company: Worley\nPrimary Location: IND-MM-Mumb...,"₹4,80,429 /yr (est.)",3.7,#N/A,#N/A,N/A
66,Data Analyst,Bobble Keyboard,Data Analyst Intern,India,Good Problem solving and Analytical skillsBasi...,"₹18,000 /mo (est.)",#N/A,#N/A,#N/A,"{'languages': ['python', 'sql'], 'frameworks':..."
67,Data Analyst,Mercedes-Benz Research and Development India P...,Data Analyst,Bangalore,Aufgaben\nEducation: Masters/Bachelors in Comp...,"₹5,76,607 /yr (est.)",4.0,#N/A,#N/A,"{'languages': ['python', 'sql'], 'frameworks':..."
68,Data Analyst,Mobile Programming India Pvt Ltd,Data Analyst,Bangalore,we need a resource urgently with dashboard/rep...,"₹10,00,000 /yr (est.)",#N/A,#N/A,#N/A,N/A
69,Data Analyst,Capgemini4.0,13097186 - BCARD Data analyst C2 Chris,Pimpri-Chinchwad,Data Analyst As a Data Analyst your main respo...,"₹4,50,791 /yr (est.)",4.0,#N/A,#N/A,"{'languages': ['r', 'python', 'sql'], 'framewo..."


In [218]:
for i in jobs["company_skills"]:
    print(i)

{'languages': ['html', 'html5', 'css', 'javascript'], 'frameworks': ['jquery', 'react', 'angular'], 'databases': [], 'skills': []}
{'languages': ['html', 'html5', 'css'], 'frameworks': ['jquery', 'bootstrap'], 'databases': [], 'skills': []}
{'languages': ['html5'], 'frameworks': ['react'], 'databases': [], 'skills': []}
{'languages': ['html', 'css', 'go', 'javascript'], 'frameworks': ['react', 'redux'], 'databases': [], 'skills': ['git']}
{'languages': ['html5', 'javascript'], 'frameworks': ['react'], 'databases': [], 'skills': []}
{'languages': ['typescript', 'css', 'javascript', 'html'], 'frameworks': ['react', 'redux'], 'databases': [], 'skills': []}
{'languages': [], 'frameworks': ['react'], 'databases': [], 'skills': []}
{'languages': ['html5', 'javascript'], 'frameworks': ['next.js', 'react'], 'databases': [], 'skills': []}
{'languages': ['html', 'css', 'javascript'], 'frameworks': ['react', 'redux'], 'databases': [], 'skills': []}
{'languages': ['html', 'java', 'c'], 'frameworks

In [136]:
decode=json.loads(final_json["React Developer"][0])

In [139]:
len(final_json["React Developer"])

60

In [123]:
data=json.loads(data["React Developer"][0])

In [104]:
data["company_skills"]

{'languages': ['css', 'php', 'html', 'sql', 'javascript'],
 'frameworks': ['Vue', 'React', 'Angular'],
 'databases': [],
 'skills': ['git']}

In [79]:
final_json["React Developer"][0]

'{"company_name": "First Longevity Limited", "job_title": "Title: Full Stack Web Developer with WordPress Experience", "location": "Remote", "job_description": "Longevity.Technology is a rapidly growing media company seeking an experienced and dedicated Full Stack Web Developer with strong WordPress skills to join our team. As a Full Stack Web Developer, you will be responsible for creating and maintaining high-quality web applications, ensuring the best possible user experience for our clients and customers.--------------------THE JOB IS FROM 10AM - 6PM UK TIME; PLEASE ONLY APPLY IF YOU CAN WORK DURING THESE HOURS. IN FUTURE IT MIGHT BECOME MORE FLEXIBLE.--------------------Responsibilities:Develop and maintain high-quality, responsive, and scalable web applications using a combination of HTML, CSS, JavaScript, PHP, and WordPress.Implement custom WordPress themes, plugins, and functionality based on project requirements and specifications.Optimize websites for speed and performance, e

In [39]:
import pandas as pd

In [49]:
dfx=pd.DataFrame(final_json["React Developer"])

In [73]:
dfx[0][59]["company_skill"]

TypeError: string indices must be integers

In [70]:
len(dfx)

60

In [72]:
for i in range(len(dfx)):
    print(dfx[0][i])

{"company_name": "First Longevity Limited", "job_title": "Title: Full Stack Web Developer with WordPress Experience", "location": "Remote", "job_description": "Longevity.Technology is a rapidly growing media company seeking an experienced and dedicated Full Stack Web Developer with strong WordPress skills to join our team. As a Full Stack Web Developer, you will be responsible for creating and maintaining high-quality web applications, ensuring the best possible user experience for our clients and customers.--------------------THE JOB IS FROM 10AM - 6PM UK TIME; PLEASE ONLY APPLY IF YOU CAN WORK DURING THESE HOURS. IN FUTURE IT MIGHT BECOME MORE FLEXIBLE.--------------------Responsibilities:Develop and maintain high-quality, responsive, and scalable web applications using a combination of HTML, CSS, JavaScript, PHP, and WordPress.Implement custom WordPress themes, plugins, and functionality based on project requirements and specifications.Optimize websites for speed and performance, en